In [1]:
import os
print(os.getcwd())
def update_working_directory():
    from pathlib import Path
    p = Path(os.getcwd()).parents[0]
    os.chdir(p)
    print(p)
update_working_directory()

/Users/admin/Projects/vocabulary_learning/notebooks
/Users/admin/Projects/vocabulary_learning


In [2]:
import numpy as np
import pandas as pd

from src.models.train_utils import *

In [3]:
vocab = initiate_vocab()

historicalData, triesSession = initiate_session()

In [4]:
#while (sum(vocab['SucceedSession']) + sum(vocab['Succeed'])) != len(vocab):
for i in range(5):
    
    nKnownWordsSession = sum(vocab['SucceedSession'])

    # Pick a random word
    rowToTest = choose_a_word(vocab)

    # Choose language
    input_language, output_language = choose_a_language()

    # Take a guess
    yourGuess = prompt_question(vocab, rowToTest, input_language, output_language, nKnownWordsSession, triesSession)

    # Answer analysis
    isItCorrect = yourGuess == vocab.loc[rowToTest,output_language]
    isItAnotherWord = yourGuess in vocab[output_language].values

    # Do we want to stop?
    if (yourGuess == "STOP IT"):
        print("Your last word was '{}={}'".format(
            vocab.loc[rowToTest,input_language], vocab.loc[rowToTest,output_language]))
        break

    # If correct answer
    if isItCorrect:
        if_correct()

    # If wrong answer
    if not isItCorrect:
        if_not_correct(vocab, rowToTest, input_language, output_language, yourGuess, isItAnotherWord)

    # Add to historical data
    historicalData = add_historical_data(historicalData, vocab, rowToTest, output_language, isItCorrect)

    # Update vocab
    vocab = update_vocab(vocab, rowToTest, isItCorrect)

    triesSession += 1

0/0 - What is the English for 'das Organ'?    the organ


Correct!


1/1 - What is the English for 'überall'?    overall


Sorry, it was 'überall = everywhere'


Write it again    überall everywhere
1/2 - What is the English for 'normal'?    normal


Correct!


2/3 - What is the English for 'erste'?    first


Correct!


3/4 - What is the German for 'the wine'?    der Wein


Correct!


In [5]:
# Update and save historical data
update_historical_data(historicalData)

# Update and save vocab
finalize_vocab(vocab)